Defining a function that takes all the information, aobut each mountain, from the table at onthesnow.com and outputs the information into an organized list of tuples.

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def grab_snow_reports():
    # grabbing the HTML and souping it with the basic python parser
    response = requests.get("https://www.onthesnow.com/colorado/skireport")
    soup = BeautifulSoup(response.text, "html.parser")

    # grabbing all the boxes of the rows inside of the table
    mountains_info_uncleaned = soup.find_all("span", class_="h4 styles_h4__x3zzi")

    # cleaning each row's boxes and adding it to mountains info cleaned
    mountains_info_cleaned = []

    ind = 0
    item = 1
    current_mountain = []
    while ind < len(mountains_info_uncleaned):
        # at the end of every row (each 5 elements) add the last element, number of open lifts, and reset the current mountain list
        if mountains_info_uncleaned[ind] and (ind + 1) % 5 == 0:
            current_mountain.append(mountains_info_uncleaned[ind].text[:-1])
            time_grabbed = str(datetime.now())
            current_mountain.append(time_grabbed)
            mountains_info_cleaned.append(tuple(current_mountain))
            current_mountain = []
            ind += 1
            item = 1

        # adding elements that are not the last element of the row
        elif mountains_info_uncleaned[ind]:
            # removing a dash (signifiying white space) at the end of the snowfall item
            if item == 2:  
                current_mountain.append(mountains_info_uncleaned[ind].text[:-1])

            # adding a space and parenthesis to the base depth information for readability
            elif item == 3:
                string = mountains_info_uncleaned[ind].text
                fixed = ""
                i = 0
                while True:
                    if string[i] == "\"":
                        fixed += string[:i + 1] + " " + "(" + string[i + 1:] + ")"
                        current_mountain.append(fixed)
                        break
                    i += 1

            # adding a space to the amount of open trails for readability
            elif item == 4:
                string = mountains_info_uncleaned[ind].text[:-5]
                fixed = ""
                # for three digit percentages
                if string[-3] == "0":
                    fixed += string[:-4] + " " + string[-4:]
                    current_mountain.append(fixed)
                # for two digit percentages
                else:
                    fixed += string[:-3] + " " + string[-3:]
                    current_mountain.append(fixed)
                    
            # all other cases (just the resort name basically)
            else:
                current_mountain.append(mountains_info_uncleaned[ind].text)
            
            # incrementing the index and current item in the row after adding an item to the current mountain info list
            ind += 1
            item += 1
    
    return mountains_info_cleaned

Writing a function that can grab the snow report based on a certain interval for a certain amount of days and output to a csv file

In [2]:
from time import sleep

def write_csv(mountain_name="all", delay=30, track_for=3):
    # amount of days to track it for
    amount_loops = (track_for * 24 * 60 * 60) // (60 * delay)

    # if they dont specify a mountain to track
    if mountain_name == "all":
        # cleaning out the files and writing in the headers for each collumn
        report = grab_snow_reports()
        for mountain in report:
            with open(f"csv_info_all_mountains/{mountain[0]}.csv", "w") as log:
                log.write("Resort Name,Snowfall 24h,Base Depth,Open Trails,Open Lifts,Date and Time Taken\n")
        
        curr_loop = 0
        while curr_loop < amount_loops:
            report = grab_snow_reports()
            for mountain in report:
                with open(f"csv_info_all_mountains/{mountain[0]}.csv", "a") as log:
                    for info_ind in range(6):
                        # adding commas between each value except not at then ends
                        if info_ind == 5:
                            log.write(mountain[info_ind])
                        else:
                            log.write(mountain[info_ind] + ",")
                    # adding a new line after each row/mountain
                    log.write("\n")
            # setting the delay so that it grabs the info after a certain time
            sleep(60 * delay)
            curr_loop += 1
    
    # if they specify a mountain to track
    else:
        # cleaning out the file and writing in the headers for each collumn
        with open(f"{mountain_name}.csv", "w") as log:
            log.write("Resort Name,Snowfall 24h,Base Depth,Open Trails,Open Lifts,Date and Time Taken\n")

        curr_loop = 0
        while curr_loop < amount_loops:
            with open(f"{mountain_name}.csv", "a") as log:
                report = grab_snow_reports()
                for mountain in report:
                    if mountain[0] == mountain_name:
                        for info_ind in range(6):
                            # adding commas between each value except not at then ends
                            if info_ind == 5:
                                log.write(mountain[info_ind])
                            else:
                                log.write(mountain[info_ind] + ",")
                    else:
                        continue
                    # adding a new line after each row/mountain
                    log.write("\n")
            # setting the delay so that it grabs the info after a certain time
            sleep(60 * delay)
            curr_loop += 1

In [3]:
write_csv("all", 60, 3)

KeyboardInterrupt: 